In [1]:
from bs4 import BeautifulSoup
import csv
from os import mkdir
from os.path import exists, join
import urllib.request as request

In [2]:
datadir = "data"
if not exists(datadir):
    mkdir(datadir)

In [4]:
if not exists("C:/Users/ashut/Desktop/thesis/edgar-extraction/tmp"):
    mkdir("C:/Users/ashut/Desktop/thesis/edgar-extraction/tmp")

In [5]:
source = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
cache = join("C:/Users/ashut/Desktop/thesis/edgar-extraction/tmp", "List_of_S%26P_500_companies.html")

In [6]:
def retrieve():
    request.urlretrieve(source, cache)

In [28]:
def extract():
    source_page = open(cache,'rb') #.read()
    soup = BeautifulSoup(source_page, "html.parser")
    table = soup.find("table", {"class": "wikitable sortable"})

    # Fail now if we haven't found the right table
    header = table.findAll("th")
    if header[0].text.rstrip() != "Symbol" or header[1].string != "Security":
        raise Exception("Can't parse Wikipedia's table!")
    
    # Retrieve the values in the table
    records = []
    symbols = []
    cik = []
    rows = table.findAll("tr")
    for row in rows:
        fields = row.findAll("td")
        if fields:
            symbol = fields[0].text.rstrip()
            # fix as now they have links to the companies on WP
            name = fields[1].text.replace(",", "")
            sector = fields[3].text.rstrip()
            cik = fields[7].text.rstrip()
            records.append([symbol, name, sector, cik])
            symbols.append(symbol + "\n")

    header = ["Symbol", "Name", "Sector", "CIK"]
    writer = csv.writer(
        open("./data/constituents.csv", "w"), lineterminator="\n"
    )
    writer.writerow(header)
    # Sorting ensure easy tracking of modifications
    records.sort(key=lambda s: s[1].lower())
    writer.writerows(records)

    with open("./data/constituents_symbols.txt", "w") as f:
        # Sorting ensure easy tracking of modifications
        symbols.sort(key=lambda s: s[0].lower())
        f.writelines(symbols)

In [25]:
def process():
    retrieve()
    extract()

In [29]:
if __name__ == "__main__":
    process()